# MyoSDK Tutorial

## What is MyoSDK?

MyoSDK is a Python library that allows you to:
- **Extract motion data** from videos (called "tracking") - it analyzes a video of a person moving and creates a digital representation of their motion
- **Retarget motion** onto 3D characters - it takes the extracted motion and applies it to a 3D character model, making the character move the same way as the person in the video

## What You'll Learn

This tutorial will walk you through the complete process:
1. Setting up the MyoSDK client
2. Exploring available 3D characters
3. Uploading a video file
4. Extracting motion data from the video (tracking)
5. Applying that motion to a 3D character (retargeting)
6. Downloading the final animated character file

## Prerequisites

Before starting, make sure you have:
- **Python 3.8 or higher** installed
- **An API key** from MyoLab (you'll need to sign up and get your API key)
- **A video file** showing a person moving (MP4 format recommended)
- **Basic Python knowledge** (understanding variables, functions, and basic syntax)

## Installation

If you haven't installed `myosdk` yet, run this command in your terminal:
```bash
pip install myosdk
```

Or if you have the wheel file in this directory:
```bash
pip install myosdk-0.1.0-py3-none-any.whl
```

In [4]:
# Step 1: Import required libraries
import os
from myosdk import Client

# Step 2: Set up your API key
# The API key is like a password that authenticates you with the MyoSDK service.
# You can get your API key from your MyoLab account dashboard at dev.myolab.ai.
# Instructions are available at https://docs.myolab.ai/docs/myosdk/getting-started/api-key

# Option 1 (Recommended): Set it as an environment variable
# In your terminal, run: export MYOSDK_API_KEY="your-api-key-here"
# Or on Windows: set MYOSDK_API_KEY=your-api-key-here

# Option 2: Replace "..." below with your actual API key (less secure, but easier for testing)
api_key = os.getenv("MYOSDK_API_KEY", "...")

# Step 3: Create the client
# The client is your connection to the MyoSDK service. It handles all communication
# with the servers. The base_url tells it which server to connect to.
client = Client(api_key=api_key, base_url="https://v2m-alb-us-east-1.myolab.ai")

# Verify the connection worked
print("Client initialized successfully.")
print("✓ You're ready to use MyoSDK!")

Client initialized successfully.
✓ You're ready to use MyoSDK!


## 1. Explore Available Characters

Before we start processing videos, let's see what 3D characters are available for retargeting.

**What is retargeting?** Retargeting means taking motion from one source (like a person in a video) and applying it to a different character (like a 3D model). Think of it like copying someone's dance moves and making a 3D character do the same dance.

**Why explore characters first?** You need to know which character you want to use before you can retarget motion onto it. Each character has a unique ID that you'll need later.

In [6]:
# List available characters
# The limit=5 parameter means we'll only show the first 5 characters
# You can increase this number or remove it to see more characters
characters = client.characters.list(limit=5)

# Display the results
print("Available characters:")
print(characters)

# Tip: Look for the 'character_id' in the output - you'll need this later!
# The output will show information about each character, including:
# - character_id: A unique identifier (like "0199e278-06e6-7726-a6ba-9929f79005e8")
# - name: The character's name
# - version: The version of the character model

AuthenticationError: Invalid or expired API key

## 2. Upload Video

Now we need to upload a video file to the MyoSDK platform. This video should show a person moving (walking, dancing, gesturing, etc.).

**Important requirements for your video:**
- **Format**: MP4 is recommended, but other common video formats work too
- **Content**: Should show a person clearly (full body or upper body)
- **Quality**: Better quality videos produce better results
- **Length**: Can be any length, but shorter videos process faster

**What happens when you upload?** The video is stored on MyoSDK's servers and gets a unique ID (called an "asset_id"). This ID is how the system references your video in later steps.

In [ ]:
# Step 1: Specify the path to your video file
# IMPORTANT: Replace this path with the actual path to your video file on your computer
# Examples:
#   - On Mac/Linux: "/Users/yourname/Videos/my_video.mp4"
#   - On Windows: "C:\\Users\\yourname\\Videos\\my_video.mp4"
#   - Or use a relative path: "videos/my_video.mp4" (if the file is in a 'videos' folder)

video_path = "/home/ouassim/dev/myoGalleryKinesis/tmp-assets/input.mp4"  # ← CHANGE THIS to your video path

# Step 2: Upload the video
# This sends your video file to MyoSDK's servers
print(f"Uploading video from: {video_path}")
print("This may take a moment depending on your video file size...")

try:
    input_asset = client.assets.upload_file(video_path)
    print("✓ Video uploaded successfully!")
    print(f"Asset ID: {input_asset['asset_id']}")
    print("This ID will be used in the next step to reference your video.")
except FileNotFoundError:
    print("❌ Error: Could not find the video file.")
    print("Please check that the path is correct and the file exists.")
except Exception as e:
    print(f"❌ Error uploading video: {e}")
    print("Please check your API key and internet connection.")

## 3. Motion Tracking

Now we'll extract motion data from your uploaded video. This process is called "tracking."

**What is tracking?** The system analyzes each frame of your video to identify where the person's body parts are (head, shoulders, arms, legs, etc.) and how they move over time. This creates a digital representation of the motion that can be applied to 3D characters.

**What are the parameters?**
- `fps`: Frames per second - how many frames per second to analyze (30 is a good default)
- `skeleton`: The type of skeleton model to use ("humanoid" works for most human motion)

**Note:** This step can take several minutes depending on your video length. The system processes your video on remote servers, so you'll need to wait for it to complete.

In [ ]:
# Start the tracking job
# This tells the MyoSDK service to begin analyzing your video for motion
print("Starting motion tracking job...")
print("This will analyze your video to extract motion data.")

# Start the tracker job
# - video_asset_id: The ID of the video we uploaded in the previous step
# - params: Configuration options for the tracking
#   * fps: 30 frames per second (standard video frame rate)
#   * skeleton: "humanoid" means it's tracking a human-like skeleton
tracker_job = client.jobs.start_tracker(
    video_asset_id=input_asset["asset_id"],  # Uses the asset ID from the upload step
    params={"fps": 30, "skeleton": "humanoid"}
)

# The job has been started - we get back a job_id to track its progress
print(f"✓ Tracker job started!")
print(f"Job ID: {tracker_job['job_id']}")
print("The job is now processing on the server. This may take a few minutes...")

In [ ]:
# Wait for the tracking job to complete
# The wait() function will periodically check the job status until it's done
# This may take several minutes depending on your video length
print("Waiting for tracker job to complete...")
print("⏳ This may take a few minutes. Please be patient...")

# Wait for the job to finish
# The wait() function blocks (pauses) until the job is complete
tracker_result = client.jobs.wait(tracker_job["job_id"])

# Check if the job succeeded
if tracker_result["status"] == "SUCCEEDED":
    # Extract the trackers asset ID from the result
    # This ID represents the motion data that was extracted
    trackers_asset_id = tracker_result["output"]["trackers_asset_id"]
    print(f"✓ Tracker job succeeded!")
    print(f"Motion data Asset ID: {trackers_asset_id}")
    print("This motion data can now be applied to a 3D character.")
else:
    # If the job failed, show the error information
    print("❌ Tracker job failed!")
    print("Error details:", tracker_result)
    print("\nCommon issues:")
    print("- Video quality too low")
    print("- Person not clearly visible in video")
    print("- Video format not supported")

## 4. Motion Retargeting

Now we'll take the motion data we extracted and apply it to a 3D character. This is called "retargeting."

**What is retargeting?** Think of it like this: you recorded someone dancing, extracted their dance moves, and now you're applying those exact same moves to a 3D character. The character will move in the same way as the person in your video.

**What you need:**
- The `trackers_asset_id` from the previous step (the motion data)
- A `character_id` from the character list we explored earlier
- A `character_version` (usually "v1.0.0" for the latest version)

**Note:** Make sure you've run the character exploration step (Section 1) to get a valid character_id!

In [7]:
# Set up the character information
# IMPORTANT: Replace the character_id below with one from the character list you saw in Section 1
# You can copy the character_id from the output of the characters.list() call above

character_id = "0199e278-06e6-7726-a6ba-9929f79005e8"  # ← Replace with a character_id from Section 1
character_version = "v1.0.0"  # Usually "v1.0.0" for the latest version

print(f"Starting retargeting job...")
print(f"Character ID: {character_id}")
print(f"Character Version: {character_version}")
print("Applying motion data to the 3D character...")

# Start the retargeting job
# This takes the motion data (trackers) and applies it to the specified character
retarget_job = client.jobs.start_retarget(
    tracker_asset_id=trackers_asset_id,  # The motion data from the tracking step
    character_id=character_id,            # Which character to animate
    character_version=character_version   # Which version of the character
)

print(f"✓ Retarget job started!")
print(f"Job ID: {retarget_job['job_id']}")
print("The job is now processing. This may take a few minutes...")

Starting retargeting job...
Character ID: 0199e278-06e6-7726-a6ba-9929f79005e8
Character Version: v1.0.0
Applying motion data to the 3D character...


NameError: name 'trackers_asset_id' is not defined

In [ ]:
# Wait for the retargeting job to complete
print("Waiting for retarget job to complete...")
print("⏳ This may take a few minutes. Please be patient...")

# Wait for the job to finish
retarget_result = client.jobs.wait(retarget_job["job_id"])

# Check if the job succeeded
if retarget_result["status"] == "SUCCEEDED":
    # Extract the motion asset ID from the result
    # This ID represents the final animated character file
    motion_asset_id = retarget_result["output"]["motion_asset_id"]
    print(f"✓ Retarget job succeeded!")
    print(f"Animated character Asset ID: {motion_asset_id}")
    print("You can now download the animated character file!")
else:
    # If the job failed, show the error information
    print("❌ Retarget job failed!")
    print("Error details:", retarget_result)
    print("\nCommon issues:")
    print("- Invalid character_id (make sure you used one from the character list)")
    print("- Motion data incompatible with character")
    print("- Character version not available")

## 5. Download Result

Finally, we'll download the generated animated character file. The file will be in GLB format, which is a 3D file format that can be used in many 3D software and game engines.

**What is a GLB file?** GLB is a binary format for 3D models that includes the character geometry, textures, and animation data all in one file. You can:
- View it in 3D viewers like Blender, Unity, Unreal Engine
- Use it in web applications with libraries like Three.js
- Import it into game engines or animation software

**Note:** Make sure the output directory exists, or the download will create it for you.

In [8]:
# Specify where to save the downloaded file
# You can change this path to wherever you want to save the file
# The .glb extension indicates it's a 3D model file with animation
output_path = "tmp-assets/tutorial_output.glb"  # ← Change this if you want a different location

# Create the directory if it doesn't exist
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

print(f"Downloading animated character to: {output_path}")
print("This may take a moment depending on file size...")

try:
    # Download the animated character file
    # This retrieves the GLB file from MyoSDK's servers and saves it to your computer
    client.assets.download(motion_asset_id, output_path)
    print("✓ Download complete!")
    print(f"Your animated character is saved at: {output_path}")
    print("\n🎉 Congratulations! You've successfully:")
    print("   1. Uploaded a video")
    print("   2. Extracted motion data")
    print("   3. Applied it to a 3D character")
    print("   4. Downloaded the result")
    print("\nYou can now use this GLB file in your 3D projects!")
except Exception as e:
    print(f"❌ Error downloading file: {e}")
    print("Please check your internet connection and try again.")

This may take a moment depending on file size...
❌ Error downloading file: name 'motion_asset_id' is not defined
Please check your internet connection and try again.


## Summary & Next Steps

Congratulations! You've completed the full MyoSDK workflow. Here's what you accomplished:

1. ✅ **Set up the MyoSDK client** - Connected to the service with your API key
2. ✅ **Explored available characters** - Found 3D characters you can animate
3. ✅ **Uploaded a video** - Sent your video to the platform
4. ✅ **Extracted motion** - Analyzed the video to get motion data
5. ✅ **Retargeted motion** - Applied the motion to a 3D character
6. ✅ **Downloaded the result** - Got your animated GLB file

### What Can You Do Next?

- **Use the GLB file in your projects:**
  - Import into Blender, Unity, or Unreal Engine
  - Use in web applications with Three.js
  - Share with others or use in games/animations

- **Experiment with different videos:**
  - Try different types of movement (dancing, sports, gestures)
  - Test with different video qualities
  - Try different characters from the character list

- **Customize the process:**
  - Adjust the `fps` parameter for different frame rates
  - Try different skeleton types if available
  - Experiment with different character versions

### Troubleshooting Tips

If you encounter issues:

1. **API Key Problems:**
   - Make sure your API key is correct
   - Check that you have an active MyoLab account
   - Verify the API key is set in your environment or code

2. **Video Upload Issues:**
   - Check that the file path is correct
   - Ensure the video file exists and is readable
   - Verify your internet connection

3. **Job Failures:**
   - Check video quality (person should be clearly visible)
   - Ensure the video format is supported
   - Verify the character_id is valid

4. **Download Issues:**
   - Make sure you have write permissions in the output directory
   - Check your internet connection
   - Verify the motion_asset_id is correct

### Getting Help

- Check the MyoSDK documentation for more details
- Review the API reference for advanced options
- Contact MyoLab support if you need assistance

Happy animating! 🎬✨